# Schnorr

In this chapter we prove knowledge of a scalar using the Schnorr identification protocol.

# What is Schnorr?

The [Schnorr identification protocol](https://en.wikipedia.org/wiki/Schnorr_signature) allows Peggy to prove her identity to Victor. Her identity is a curve point (public key). She proves her identity using the discrete logarithm of this point (secret key) which is a scalar.

# What is the proof?

Peggy and Victor are engaged in an interactive proof.

There is a curve point (public key).

Peggy (thinks she) knows the discrete logarithm of this point (secret key). She wants to convince Victor of this fact without revealing the logarithm.

Victor is sceptical and wants to see evidence. He wants to expose Peggy as a liar if her logarithm doesn't match the curve point.

Peggy wins if she convinces Victor. Victor wins by accepting only matching logarithms.

# Set up Jupyter

Run the following snippet to set up your Jupyter notebook for the workshop.

In [ ]:
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

# Import here so cells don't depend on each other
from IPython.display import display
from typing import List, Tuple, Dict
import ipywidgets as widgets
import random

from local.ec.static import Scalar, CurvePoint, ONE_POINT
import local.stats as stats

# Select the scenario

Choose the good or the evil scenario. See how it affects the other cells further down.

1. **Peggy is honest** 😇 She knows the logarithm. She wants to convince Victor of a true statement.
2. **Peggy is lying**  😈 She doesn't actually know anything! She tries to fool Victor into believing a false statement.

In [ ]:
# You can adjust the selection any time

def generate_keys(values: Dict[str, bool]):
    global P, x
    
    x = Scalar.random()
    if values["new"]:
        # Good: x is the logarithm of P
        P = ONE_POINT * x
    else:
        # Evil: x is (likely) not the logarithm of P
        P = ONE_POINT * Scalar.random()

matching_dropdown = widgets.Dropdown(
    options=[
        ("Peggy knows the logarithm 😇", True),
        ("Peggy knows nothing 😈", False)],
    value=True,
    description="Scenario:",
)
matching_dropdown.observe(generate_keys, names="value")

# Generate default keys
generate_keys({"new": matching_dropdown.value})
# Display dropdown
display(matching_dropdown)

# How the proof roughly goes

Victor knows a point $P$. Peggy knows a scalar. She wants to prove that her scalar is the discrete logarithm of the point.

1. Peggy sends a random point to Victor.
1. Victor sends a random scalar to Peggy.
1. Peggy computes a scalar from the random values which were exchanged and from the discrete logarithm of $P$. Peggy sends the scalar to Victor.
1. Victor verifies that Peggy computed the scalar correctly.

# Why the proof works

The scalar that Peggy has to compute requires knowledge of the discrete logarithm of $P$. It is exponentially unlikely that Peggy computes a scalar that passes Victor's check without this knowledge. If Peggy's scalar passes Victor's check, he is confident that Peggy must know the discrete logarithm ✅

The random point that Peggy sends (together with its discrete logarithm) serve as blinding factors. This makes sure that Victor sees random noise ✅

# How the proof precisely goes

[A useful flow chat can be found online.](https://www.zkdocs.com/docs/zkdocs/zero-knowledge-protocols/schnorr/)

Victor knows the point $P$. Peggy knows a scalar $x$. She wants to prove that $P = I * x$ where $I$ is the one-point.

1. Peggy generates a random scalar $r$ (nonce).
1. Peggy sends $R = I *r $ (nonce point) to Victor.
1. Victor sends a random scalar $e$ (challenge) to Peggy.
1. Peggy sends the scalar $s = r + e * x$ (response) to Victor.
1. Victor verifies that the equation $I * s \overset{?}{=} R + P * e$ holds.

In [ ]:
class Peggy:
    def __init__(self, x: Scalar):
        """
        0. Give Peggy her scalar x.
        """
        self.x = x
        
    def commit(self) -> CurvePoint:
        """
        1. Peggy generates a random scalar r.
        
        2. Peggy computes the point R = I * r and sends it to Victor.
        """
        self.r = Scalar.random()
        R = ONE_POINT * self.r
        return R
    
    def respond(self, e: Scalar) -> Scalar:
        """
        4. Peggy responds by sending the scalar s = r + e * x to Victor.
        """
        s = self.r + e * self.x
        return s

class Victor:
    def __init__(self, P: CurvePoint):
        """
        0. Give Victor his point P.
        """
        self.P = P
    
    def challenge(self, R: CurvePoint) -> Scalar:
        """
        3. Victor challenges Peggy with a random scalar e.
        """
        self.R = R
        self.e = Scalar.random()
        return self.e
    
    def verify(self, s: Scalar) -> bool:
        """
        5. Victor verifies that the equation I * s =? R + P * e holds.
        """
        return ONE_POINT * s == self.R + self.P * e

# Run the proof

Let's see the proof in action.

Run the Python code below and see what happens. The outcome depends on the scenario you picked. The outcome is also randomly different each time. Feel free to run the code multiple times!

In [ ]:
# Feel free to run this multiple times

peggy = Peggy(x)
victor = Victor(P)

R = peggy.commit()
print(f"R = {R}")

e = victor.challenge(R)
print(f"e = {e}")

s = peggy.respond(e)
print(f"s = {s}")
print()

# Victor is convinced
if victor.verify(s):
    # Matching secret key (good)
    if matching_dropdown.value:
        print("Convinced 👌 (expected)")
    # Different secret key (evil)
    else:
        print("Convinced 👌 (Victor was fooled)")
# Victor is not convinced
else:
    # Matching secret key (good)
    if matching_dropdown.value:
        print("Not convinced... 🤨 (Peggy was dumb)")
    # Different secret key (evil)
    else:
        print("Not convinced... 🤨 (expected)")

# How the proof is complete

If Peggy knows the discrete logarithm, then **Victor will always be convinced** by her proof.

This is because Peggy is always able to compute a scalar that passes Victor's check.

Let's run a couple of exchanges and see how they go.

In [ ]:
n_exchanges_complete_slider = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
n_exchanges_complete_slider

In [ ]:
# Good scenario:
# Peggy knows the discrete logarithm
x2 = Scalar.random()
P2 = ONE_POINT * x2

honest_peggy = Peggy(x2)
victor = Victor(P2)

peggy_success = 0

for _ in range(n_exchanges_complete_slider.value):
    R = honest_peggy.commit()
    e = victor.challenge(R)
    s = honest_peggy.respond(e)

    if victor.verify(s):
        peggy_success += 1
        
peggy_success_rate = peggy_success / n_exchanges_complete_slider.value * 100

print(f"Running {n_exchanges_complete_slider.value} exchanges")
print(f"Honest Peggy wins {peggy_success_rate:0.2f}% of the time")
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest")

# How the proof is sound

If Peggy doesn't know the discrete logarithm, then **Victor will almost always reject** her proof.

It is exponentially unlikely that she finds a scalar $s$ that satisfies Victor's equation. Finding $s$ amounts to finding the discrete logarithm of $P$ because the discrete logarithm can be computed directly from $s$ and the blinding factors.

The beautiful thing about Schnorr is that the proof is secure after a **single round**. This makes the proof very short.

In most interactive proofs, Peggy and Victor must repeat their exchange a couple of rounds to increase security.

Let's run a couple of exchanges and see how they go.

In [ ]:
n_exchanges_sound_slider = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
n_exchanges_sound_slider

In [ ]:
# Evil scenario:
# Peggy doesn't know the discrete logarithm
x3 = Scalar.random()
P3 = ONE_POINT * Scalar.random()

lying_peggy = Peggy(x3)
victor = Victor(P3)

victor_success = 0

for _ in range(n_exchanges_sound_slider.value):
    R = lying_peggy.commit()
    e = victor.challenge(R)
    s = lying_peggy.respond(e)

    if not victor.verify(s):
        victor_success += 1

victor_success_rate = victor_success / n_exchanges_sound_slider.value * 100

print(f"Running {n_exchanges_sound_slider.value} exchanges")
print(f"Victor wins against lying Peggy {victor_success_rate:0.2f}% of the time")
print()

if victor_success_rate < 75:
    print("Victor has a small chance of getting fooled which decreases with the curve size")
else:
    print("It is basically impossible to fool Victor")

# How the proof is zero-knowledge

The proof itself looks like random noise. Nothing can be extracted from this noise.

Everything that is sent over the wire is randomized:

1. Peggy sends a random point.
1. Victor sends a random scalar.
1. Peggy sends a scalar which depends on the first two values. This scalar includes a random blinding factor.

The transcripts follow a pattern: Two values are random and one value is completely determined by the first two. Two variables and one equation.

We can replicate this pattern:

1. Compute a random scalar $e$
1. Compute a random scalar $s$
1. Compute the point $R$ from the other values to satisfy the Victor's equation $I * s = R + P * e$.

We transform Victor's equation to obtain $R = I * s - P * s$. This is what we compute.

Let's run a chi-square test to see if the original transcripts are distinguishable from the fake transcripts.

In [ ]:
n_transcripts_slider = widgets.IntSlider(min=1000, max=50000, value=10000, step=1000, description="#Transcripts")
n_transcripts_slider

In [ ]:
peggy = Peggy(x)
victor = Victor(P)

def real_transcript() -> Tuple:
    R = peggy.commit()
    e = victor.challenge(R)
    s = peggy.respond(e)
    return R, e, s


def fake_transcript() -> Tuple:
    e = Scalar.random()
    s = Scalar.random()
    R = ONE_POINT * s - P * e
    return R, e, s


print("Real transcript: {}".format(real_transcript()))
print("Fake transcript: {}".format(fake_transcript()))
print()

real_samples = [real_transcript() for _ in range(n_transcripts_slider.value)]
fake_samples = [fake_transcript() for _ in range(n_transcripts_slider.value)]

# The chi-square test is only valid if most bins are filled
# Increase the number of transcripts if there are too many empty bins

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")

stats.plot_comparison(real_samples, fake_samples, "real", "fake")